# Canvas Batch Change Dates

This script allows you to adjust the start and end times of multiple courses at once in Canvas. At each stage you will be asked to give certain inputs and asked confirmation steps before moving forward. Please read through all of the steps and confirm that you have all of the information needed.

If any of the steps along the way fail, the script will not move forward. There should be indicators of why the failure occurred and steps to take to fix the errors.

There are only two places where you might need to change text in the 'cells' below. The API_URL below should be your institution url. Near the end there is a section 'Get Course List for Changes' - you can also change the name of the csv with the courses (although I recommend naming your file 'start_end_courses.csv' for consistency).

To Run:
Once you have confirmed all of the steps

Kernel -> Run All

## 1. Token Validation
This is the token validation stage. If you do not enter a valid token, nothing else will happen.

In [1]:
from canvas_batch_change_dates import initialize

initialize()

Enter token: ········

Token Valid

Hello Alison


## 1. b) Create a course file from subaccount

In [2]:
from canvas_batch_change_dates import create_courses_file

account = create_courses_file()

Enter subaccount_id: ········


In [15]:
import pandas as pd
from settings import CANVAS

course_info = []

for c in courses:
    course_dict = {'course_id': c.id,
        'course_name': c.name,
        'term': c.enrollment_term_id,
        'course_subaccount': c.account_id,
        'course_subaccount_name': CANVAS.get_account(c.account_id).name,
        'start_at' : c.start_at,
        'end_at' : c.end_at,
        'timezone' : c.time_zone}
    
    course_info.append(course_dict)
    
df = pd.DataFrame(course_info)
df.head()

,course_id,course_name,term,course_subaccount,course_subaccount_name,start_at,end_at,timezone
0,1335,BAPA 560 300 Foundations of Managerial and Bus...,5,477,PMBA - Alumni,2018-03-13T03:26:00Z,2018-07-01T07:00:00Z,America/Los_Angeles
1,1406,UBC Professional MBA,2,479,PMBA – Class of 2023,2017-10-06T16:00:00Z,2020-08-01T07:00:00Z,America/Los_Angeles
2,2364,Professional Residency I,5,477,PMBA - Alumni,2017-11-23T19:55:00Z,2018-12-04T07:59:00Z,America/Los_Angeles
3,2586,BAAC 550 300 Foundations in Accounting - LOUIE,5,477,PMBA - Alumni,2018-01-06T21:29:00Z,None,America/Los_Angeles
4,2587,BABS 550 300 Application of Statistics in Mana...,5,477,PMBA - Alumni,2018-01-01T08:00:00Z,2018-04-30T07:00:00Z,America/Los_Angeles


In [ ]:
df.to_csv("subaccount 458 course information with start and end dates.csv")

## 2. Fill out Input CSV
1. You should have a csv called `courses_start_end.csv` in `data/input` that has the columns **course_id**, **start_date**, and **end_date** with a list of courses that you want to apply changes to.
1. Please ensure the dates are formatted as **YYYY-MM-DD** or **YYYY-MM-DD HH:MM** in your file. If you have used Excel, the columns should be formatted as a "Short Date" (YYYY-MM-DD) or "Long Date" (YYYY-MM-DD HH:MM). 
    * The script assumes that the dates are for Vancouver
    * The start_date will assume the time of start is 12:00am that day
    * The end_date will assume the time of end is 11:59pm that day


Once you have completed the script, **start_end_courses.csv** will be copied to the `/completed` folder and timestamped in the filename.


## 3. Set Participation Dates

The default setting for restricting participations to course dates is **None**. This will keep whatever setting was initially established. You can change this value below to **True** or **False** 

**None** will keep settings as is. 
**True** will restrict participation to within the given dates.
**False** will allow participation outside of the given dates. 

In [ ]:
# restrict_participation_to_dates can be True, False, or None
restrict_participation_to_dates = True

## 4. Apply the Changes
In this step you will be asked to confirm the csv that is being read, and to confirm the start and end date changes that will occur.
You should see indicators for success or failure for each of the courses as the script runs.

In [ ]:
from datetime import datetime
from canvas_batch_change_dates import get_courses_df, create_course_update_df

# NOW is the time you made the update, not the time the csv is output
NOW = datetime.now()

courses_df = get_courses_df('data/input/start_end_courses.csv')

details = create_course_update_df(courses_df, restrict_participation_to_dates)

print('\nTotal Courses: {}\n'.format(len(details)))
display(details)

## 5. Generate Final Output
The final output will be a csv with the changes that have taken place (if confirmed in the step above).

In [ ]:
from canvas_batch_change_dates import output_csv

output_csv(details)